# <center> Process of cleaning and analyzing horse racing data from Kaggle dataset "Horses for Courses"</center>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare, ttest_ind

%matplotlib inline

#Supresses scientific notation
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
#There are a lot of issues with the 'position_two' column, so I left it out.
#Furthermore, the 'position_again' column is much more consistent and has all relevant win/place information  

fields = ["position_again","bf_odds","venue_name","date","market_name","condition","barrier","handicap_weight","last_five_starts","prize_money","sex","age","jockey_sex","days_since_last_run","overall_starts","overall_wins","overall_places","track_starts","track_wins","track_places","firm_starts","firm_wins","firm_places","good_starts","good_wins","good_places","slow_starts","slow_wins","slow_places","soft_starts","soft_wins","soft_places","heavy_starts","heavy_wins","heavy_places","distance_starts","distance_wins","distance_places"]
# will use "jockey","trainer" later

df = pd.read_csv("horses.csv", skipinitialspace=True, usecols=fields, low_memory=False)

df.head()

,position_again,bf_odds,venue_name,date,market_name,condition,barrier,handicap_weight,last_five_starts,prize_money,...,slow_places,soft_starts,soft_wins,soft_places,heavy_starts,heavy_wins,heavy_places,distance_starts,distance_wins,distance_places
0,1.00,2.88,Echuca,2016-06-27,R3 1200m Mdn,HVY9,3.00,58.50,f3,2160.00,...,1.00,1.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00
1,2.00,15.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,10.00,58.50,x80x2,21175.00,...,1.00,2.00,0.00,1.00,1.00,0.00,0.00,1.00,0.00,1.00
2,3.00,95.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,6.00,56.50,79x00,28855.00,...,0.00,5.00,0.00,0.00,3.00,0.00,0.00,4.00,0.00,0.00
3,nan,20.00,Echuca,2016-06-27,R3 1200m Mdn,HVY9,2.00,56.50,f0,1475.00,...,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
4,nan,2.74,Echuca,2016-06-27,R3 1200m Mdn,HVY9,7.00,56.50,f4x,1215.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


In [3]:
df.date = pd.to_datetime(df.date, format='%Y'+'-'+'%m'+'-'+'%d')

#removes numbers from end of 'condition' strings
df.condition = df.condition.str.replace('\d+', '')

#renaming condition values so that they're uniform
df.condition = df.condition.str.replace('HVY', 'HEAVY')
df.condition = df.condition.str.replace('AWT', 'GOOD') #AWT equates to a Good surface under some weather conditions

#reverses 'last_five_starts' (originally written right-to-left) so that it's easier to read in the future
df.last_five_starts = df.last_five_starts.str[::-1]

## Creating general and track-condition-specific 'win_percent' and 'place_percent' columns:

In [4]:
#creates overall, track, and distance win_percent and place_percent columns and drops existing wins and places columns

columns_list = ["overall","track","distance"]

for x in columns_list:
    df[x+"_win_percent"] = df[x+"_wins"]/df[x+"_starts"]
    
    df[x+"_place_percent"] = df[x+"_places"]/df[x+"_starts"]

    # dropping various columns, though 'starts' columns will be used later
    df.drop([x+'_wins', x+'_places'], axis=1, inplace=True)

In [5]:
#creates a condition_starts ,condition_win_percent, and condition_place_percent column for each horse according to the condition of the track for that race

df.loc[df.condition.isna(), "condition_win_percent"] = np.nan

condition_list = ["firm","good","slow","soft","heavy"]

for x in condition_list: 
    df.loc[df.condition.str.lower() == x, "condition_starts"] = df[x+"_starts"]
    
    df.loc[df.condition.str.lower() == x, "condition_win_percent"] = df[x+"_wins"]/df[x+"_starts"]
    
    df.loc[df.condition.str.lower() == x, "condition_place_percent"] = df[x+"_places"]/df[x+"_starts"]
    
    df.drop([x+'_wins', x+'_places'], axis=1, inplace=True)

# Condition column is not necessary now that there are condition win and place percent columns 
df.drop('condition', axis=1, inplace=True)

# Replaces infinity (zero division) with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)

## Cleaning data by removing races with missing win and/or place values in 'position_again' column:

In [6]:
#Used groupby to create indices by which to sort the re-indexed dataframes below, like df_indexed and df_cleaned
df_grouped = df.groupby(['date','venue_name','market_name'])

#Drops all groups/races in 'position_again' column where sum of values [1st, 2nd, 3rd] don't add to 3 or 6
#i.e. 1+2 and 1+2+3
index_list1 = df_grouped.position_again.sum(dropna=False).where(lambda x:(x == 3) | (x == 6)).dropna().index

df_indexed = df.set_index(['date','venue_name','market_name'])

df_cleaned = df_indexed.loc[index_list1].drop_duplicates()

In [7]:
df_grouped = df_cleaned.groupby(['date','venue_name','market_name'])

#Eliminates remaining errors in 'position_again' column by making sure that there isn't a single 3rd-place finish
index_list2 = df_grouped.position_again.value_counts(normalize=True).where(lambda x:x != 1).dropna().index.droplevel('position_again')

In [8]:
df_cleaned = df_cleaned.loc[index_list2].drop_duplicates()

df_grouped = df_cleaned.groupby(['date','venue_name','market_name'])

df_cleaned.head()

position_again  bf_odds  barrier  \
date       venue_name market_name                                        
2016-06-27 Echuca     R10 1400m Hcap            1.00    18.50    12.00   
                      R10 1400m Hcap            2.00    16.50     8.00   
                      R10 1400m Hcap            3.00    11.50     1.00   
                      R10 1400m Hcap             nan    19.00     5.00   
                      R10 1400m Hcap             nan     7.60     7.00   

                                      handicap_weight last_five_starts  \
date       venue_name market_name                                        
2016-06-27 Echuca     R10 1400m Hcap            57.50            86x01   
                      R10 1400m Hcap            59.00            6378x   
                      R10 1400m Hcap            58.50            430x5   
                      R10 1400m Hcap            58.50            44460   
                      R10 1400m Hcap            58.50            411x3   

                                      prize_money      sex  age jockey_sex  \
date       venue_name market_name                                            
2016-06-27 Echuca     R10 1400m Hcap     71295.00     Mare 6.00       Male   
                      R10 1400m Hcap     34345.00  Gelding 5.00       Male   
                      R10 1400m Hcap     55655.00  Gelding 5.00       Male   
                      R10 1400m Hcap    154361.00  Gelding 6.00       Male   
                      R10 1400m Hcap     29190.00     Mare 4.00       Male   

                                      days_since_last_run  ...  \
date       venue_name market_name                          ...   
2016-06-27 Echuca     R10 1400m Hcap                16.00  ...   
                      R10 1400m Hcap                18.00  ...   
                      R10 1400m Hcap                13.00  ...   
                      R10 1400m Hcap                16.00  ...   
                      R10 1400m Hcap                13.00  ...   

                                      distance_starts  overall_win_percent  \
date       venue_name market_name                                            
2016-06-27 Echuca     R10 1400m Hcap            12.00                 0.15   
                      R10 1400m Hcap             9.00                 0.17   
                      R10 1400m Hcap            10.00                 0.10   
                      R10 1400m Hcap            16.00                 0.10   
                      R10 1400m Hcap             3.00                 0.33   

                                      overall_place_percent  \
date       venue_name market_name                             
2016-06-27 Echuca     R10 1400m Hcap                   0.15   
                      R10 1400m Hcap                   0.17   
                      R10 1400m Hcap                   0.30   
                      R10 1400m Hcap                   0.16   
                      R10 1400m Hcap                   0.33   

                                      track_win_percent  track_place_percent  \
date       venue_name market_name                                              
2016-06-27 Echuca     R10 1400m Hcap               0.30                 0.20   
                      R10 1400m Hcap               0.00                 1.00   
                      R10 1400m Hcap               0.00                 0.29   
                      R10 1400m Hcap               0.50                 0.00   
                      R10 1400m Hcap                nan                  nan   

                                      distance_win_percent  \
date       venue_name market_name                            
2016-06-27 Echuca     R10 1400m Hcap                  0.25   
                      R10 1400m Hcap                  0.22   
                      R10 1400m Hcap                  0.20   
                      R10 1400m Hcap                  0.19   
                      R10 1400m Hcap                  0.33   

     

# Creating new features and dropping others:

## Creating a weight_minus_average column:

In [9]:
average_weight = df_grouped.handicap_weight.transform('mean')

df_cleaned.insert(4,'average_weight',average_weight)

df_cleaned.insert(4,'weight_minus_average', df_cleaned.handicap_weight - df_cleaned.average_weight)

In [10]:
df_cleaned.drop('average_weight', axis=1, inplace=True)

In [11]:
df_grouped = df_cleaned.groupby(['date','venue_name','market_name'])

## Creating a prize_money_per_start_minus_average column:

### This may be one of the best indicators, as prize money is also an indicator of the difficulty of past races. Therefore, the value (meaningfulness) of past wins is taken into consideration.

In [12]:
#creates prize_money_per_start column
df_cleaned.insert(6,'prize_money_per_start', df_cleaned.prize_money/df_cleaned.overall_starts)

#Creates average_prize_money_per_start column
average_prize_money_per_start = df_grouped.prize_money_per_start.transform('mean')

df_cleaned.insert(7,'average_prize_money_per_start', average_prize_money_per_start)

#Creates prize_money_per_start_minus_average column
df_cleaned.insert(8,'prize_money_per_start_minus_average', df_cleaned.prize_money_per_start - df_cleaned.average_prize_money_per_start)

In [13]:
#drops irrelevant columns
df_cleaned.drop(['prize_money', 'prize_money_per_start', 'average_prize_money_per_start'], axis=1, inplace=True)

In [14]:
df_grouped = df_cleaned.groupby(['date','venue_name','market_name'])

### I was considering using a horse age_minus_average column, but I realized that age difference is not as good of an indicator as pure age. This is because the difference between a 2-year-old and a 4-year-old might mean little, but the difference between a 4 and 6-year-old might be a lot.

In [15]:
# Replaces infinity (zero division) with NaN
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

df_cleaned.sort_index(inplace=True)

df_grouped = df_cleaned.groupby(['date','venue_name','market_name'])

df_cleaned.head()

position_again  bf_odds  barrier  \
date       venue_name market_name                                        
2016-06-27 Echuca     R10 1400m Hcap            1.00    18.50    12.00   
                      R10 1400m Hcap            2.00    16.50     8.00   
                      R10 1400m Hcap            3.00    11.50     1.00   
                      R10 1400m Hcap             nan    19.00     5.00   
                      R10 1400m Hcap             nan     7.60     7.00   

                                      handicap_weight  weight_minus_average  \
date       venue_name market_name                                             
2016-06-27 Echuca     R10 1400m Hcap            57.50                 -0.62   
                      R10 1400m Hcap            59.00                  0.88   
                      R10 1400m Hcap            58.50                  0.38   
                      R10 1400m Hcap            58.50                  0.38   
                      R10 1400m Hcap            58.50                  0.38   

                                     last_five_starts  \
date       venue_name market_name                       
2016-06-27 Echuca     R10 1400m Hcap            86x01   
                      R10 1400m Hcap            6378x   
                      R10 1400m Hcap            430x5   
                      R10 1400m Hcap            44460   
                      R10 1400m Hcap            411x3   

                                      prize_money_per_start_minus_average  \
date       venue_name market_name                                           
2016-06-27 Echuca     R10 1400m Hcap                             -1358.43   
                      R10 1400m Hcap                             -1238.46   
                      R10 1400m Hcap                             -1317.80   
                      R10 1400m Hcap                               878.84   
                      R10 1400m Hcap                               764.45   

                                          sex  age jockey_sex  ...  \
date       venue_name market_name                              ...   
2016-06-27 Echuca     R10 1400m Hcap     Mare 6.00       Male  ...   
                      R10 1400m Hcap  Gelding 5.00       Male  ...   
                      R10 1400m Hcap  Gelding 5.00       Male  ...   
                      R10 1400m Hcap  Gelding 6.00       Male  ...   
                      R10 1400m Hcap     Mare 4.00       Male  ...   

                                      distance_starts  overall_win_percent  \
date       venue_name market_name                                            
2016-06-27 Echuca     R10 1400m Hcap            12.00                 0.15   
                      R10 1400m Hcap             9.00                 0.17   
                      R10 1400m Hcap            10.00                 0.10   
                      R10 1400m Hcap            16.00                 0.10   
                      R10 1400m Hcap             3.00                 0.33   

                                      overall_place_percent  \
date       venue_name market_name                             
2016-06-27 Echuca     R10 1400m Hcap                   0.15   
                      R10 1400m Hcap                   0.17   
                      R10 1400m Hcap                   0.30   
                      R10 1400m Hcap                   0.16   
                      R10 1400m Hcap                   0.33   

                                      track_win_percent  track_place_percent  \
date       venue_name market_name                                              
2016-06-27 Echuca     R10 1400m Hcap               0.30                 0.20   
                      R10 1400m Hcap               0.00                 1.00   
                      R10 1400m Hcap               0.00                 0.29   
                      R10 1400m Hcap               0.50                 0.00   
                      R10 1400m Hcap                nan          

# Testing which features are significant:

### general_win_indices will be used when the feature has only distinct values:

In [ ]:
general_win_indices = df_grouped.position_again.apply(lambda x: x == 1)

## For horse gender:

In [ ]:
#Removes races where only one horse gender is represented
horse_gender_population = df_grouped.sex.value_counts(normalize=True,dropna=False).where(lambda x: x != 1).dropna()

horse_gender_population_indices = horse_gender_population.index.droplevel('sex')

horse_gender_population_cleaned = df_cleaned.loc[horse_gender_population_indices].drop_duplicates()

In [ ]:
#General percentage of horse genders for races where multiple genders are represented  
horse_gender_population_cleaned.sex.value_counts(dropna=False,normalize=True).sort_values(ascending=False).drop('Unknown')

In [ ]:
horse_gender_win_indices = horse_gender_population_cleaned.position_again.apply(lambda x: x == 1)

horse_gender_population_cleaned[horse_gender_win_indices].sex.value_counts(dropna=False,normalize=True).sort_values(ascending=False).drop('Unknown')

In [ ]:
horse_sex_pop = horse_gender_population_cleaned.sex.value_counts(dropna=False,normalize=True).sort_values(ascending=False).drop('Unknown')

horse_sex_win = horse_gender_population_cleaned[horse_gender_win_indices].sex.value_counts(dropna=False,normalize=True).sort_values(ascending=False).drop('Unknown')

horse_sex_percent_difference = (horse_sex_win - horse_sex_pop)/horse_sex_pop

In [ ]:
horse_sex_percent_difference

In [ ]:
index = ['Gelding', 'Mare', 'Filly','Colt', 'Horse']

df1 = pd.DataFrame({'Total Proportion': horse_sex_pop,'Win Proportion': horse_sex_win , 'Percent Difference': horse_sex_percent_difference}, index=index)

ax = df1.plot.bar(rot=0,title='The Significance of Horse Gender')

### Using the Pearson's chi-squared test for both ways of finding population, I find horse gender isn't significant:

In [ ]:
observed1 = horse_gender_population_cleaned.sex.value_counts().sort_values(ascending=False).drop('Unknown').values
expected_percentages1 = horse_sex_pop.values
expected1 = [x*observed1.sum() for x in expected_percentages1]

test_stat1, p_value1 = chisquare(observed1, expected1)

test_stat1, p_value1

## For horse age:

In [ ]:
#Removing races where there is only one age
age_population = df_grouped.age.value_counts(normalize=True,dropna=False).where(lambda x: x != 1).dropna()

age_population_indices = age_population.index.droplevel('age')

age_population_cleaned = df_cleaned.loc[age_population_indices].drop_duplicates()

In [ ]:
age_population_cleaned.age.value_counts(normalize=True,dropna=False).sort_index()

In [ ]:
age_win_indices = age_population_cleaned.position_again.apply(lambda x: x == 1)

age_population_cleaned[age_win_indices].age.value_counts(dropna=False,normalize=True).sort_index()

In [ ]:
age_pop = age_population_cleaned.age.value_counts(dropna=False,normalize=True).sort_index()

age_win = age_population_cleaned[age_win_indices].age.value_counts(dropna=False,normalize=True).sort_index()

age_percent_difference = (age_win - age_pop)/age_pop

In [ ]:
age_percent_difference

In [ ]:
index = age_percent_difference.index

df2 = pd.DataFrame({'Total Proportion': age_pop,'Win Proportion': age_win , 'Percent Difference': age_percent_difference}, index=index)

ax = df2.plot.bar(rot=0, title='The Significance of Horse Age')

### Using the Pearson's chi-squared test, I find horse age is significant:

In [ ]:
observed2 = age_population_cleaned[age_win_indices].age.value_counts().sort_index().values
expected_percentages2 = age_pop.values
expected2 = [x*observed2.sum() for x in expected_percentages2]

test_stat2, p_value2 = chisquare(observed2, expected2)

test_stat2, p_value2

## For handicap_weight:

In [ ]:
#Removing races where there is only one age
weight_population = df_grouped.handicap_weight.value_counts(normalize=True,dropna=False).where(lambda x: x != 1).dropna()

weight_population_indices = weight_population.index.droplevel('handicap_weight')

weight_population_cleaned = df_cleaned.loc[weight_population_indices].drop_duplicates()

In [ ]:
weight_population_cleaned.handicap_weight.describe()

In [ ]:
weight_win_indices = weight_population_cleaned.position_again.apply(lambda x: x == 1)

weight_population_cleaned[weight_win_indices].handicap_weight.describe()

In [ ]:
data7a = weight_population_cleaned.handicap_weight.dropna().values

data7b = weight_population_cleaned[weight_win_indices].handicap_weight.dropna().values

plt.title("Winner and Race Distributions of Weight from Race Average", fontsize=15)
plt.hist(data7a, density=True, bins=13, range=(52,65), label='Race Average', color='b', alpha=.5, edgecolor='k')
plt.hist(data7b, density=True, bins=13, range=(52,65), label='Winner Average', color='r', alpha=.5, edgecolor='k')
plt.legend(loc='upper right')
plt.xlabel('Weight from Average')
plt.ylabel('Probability');

### Using a 2-sample T-test, again I find that handicap_weight is significant:

In [ ]:
test_stat7, p_value7 = ttest_ind(data7a, data7b)

test_stat7, p_value7

## For handicap_weight, using handicap weight_minus_average:

In [ ]:
#I use the handicap_weight filters since the same races will be eliminated
weight_population_cleaned.weight_minus_average.describe()

In [ ]:
#average weight difference for winners
weight_population_cleaned[weight_win_indices].weight_minus_average.describe()

In [ ]:
data2a = weight_population_cleaned.weight_minus_average.dropna().values

data2b = weight_population_cleaned[weight_win_indices].weight_minus_average.dropna().values

plt.title("Winner and Race Distributions of Weight from Race Average", fontsize=15)
plt.hist(data2a, density=True, bins=30, range=(-7.5,7.5), label='Race Average', color='b', alpha=.5, edgecolor='k')
plt.hist(data2b, density=True, bins=30, range=(-7.5,7.5), label='Winner Average', color='r', alpha=.5, edgecolor='k')
plt.legend(loc='upper right')
plt.xlabel('Weight from Average')
plt.ylabel('Probability');

### Using a 2-sample T-test, I find that handicap weight_minus_average is significant:

In [ ]:
test_stat2, p_value2 = ttest_ind(data2a, data2b)

test_stat2, p_value2

## For prize money, using prize_money_per_start_minus_average:

In [ ]:
df_cleaned.prize_money_per_start_minus_average.describe()

In [ ]:
#Winner prize money 
df_cleaned[general_win_indices].prize_money_per_start_minus_average.describe()

In [ ]:
data3a = df_cleaned.prize_money_per_start_minus_average.dropna().values
data3b = df_cleaned[general_win_indices].prize_money_per_start_minus_average.dropna().values

plt.title("Winner and Race Distributions of Prize Money per Start Minus Average", fontsize=15)
plt.hist(data3a, density=True, bins=40, range=(-10000,10000), label='Race Average', color='b', alpha=.6, edgecolor='k')
plt.hist(data3b, density=True, bins=40, range=(-10000,10000), label='Winner Average', color='r', alpha=.5, edgecolor='k')
plt.legend(loc='upper right')
plt.xlabel('Prize Money per Start Minus Average')
plt.ylabel('Probability');

### Using a 2-sample T-test, I find that prize money per start is significant:


In [ ]:
test_stat3, p_value3 = ttest_ind(data3a, data3b)

test_stat3, p_value3

## For overall wins:

In [ ]:
overall_pop = df_cleaned.overall_win_percent.value_counts(normalize=True,bins=10).sort_index()

overall_win = df_cleaned[general_win_indices].overall_win_percent.value_counts(normalize=True,bins=10).sort_index()

overall_percent_difference = (overall_win - overall_pop)/overall_pop

In [ ]:
overall_percent_difference

In [ ]:
index = overall_percent_difference.index

df5 = pd.DataFrame({'Total Proportion': overall_pop,'Win Proportion': overall_win , 'Percent Difference': overall_percent_difference}, index=index)

ax = df5.plot.bar(title='The Significance of Horse overall')

### There is high variance in the 100% column (aka beginner's luck). How many races before the 100% column is properly represented? That is, how many races is considered statistically significant?

### 90-100% column should be roughly 3.90 or 390%

## For Barrier:

In [ ]:
barrier_pop = df_cleaned.barrier.value_counts(normalize=True).sort_index().drop([18.00,19.00,20.00])

barrier_win = df_cleaned[general_win_indices].barrier.value_counts(normalize=True).sort_index().drop(18.00)

barrier_percent_difference = (barrier_win - barrier_pop)/barrier_pop

In [ ]:
barrier_pop

In [ ]:
barrier_win

In [ ]:
barrier_percent_difference

In [ ]:
index = barrier_percent_difference.index

df3 = pd.DataFrame({'Total Proportion': barrier_pop,'Win Proportion': barrier_win , 'Percent Difference': barrier_percent_difference}, index=index)

ax = df3.plot.bar(rot=0, title='The Significance of Barrier')

#why is barrier 1 overrepresented? Is there a problem with the data?

### Using the Pearson's chi-squared test, I find that barrier is significant:

In [ ]:
observed3 = df_cleaned[general_win_indices].barrier.value_counts().sort_index().drop(18.00).values
expected_percentages3 = barrier_pop.values
expected3 = [x*observed3.sum() for x in expected_percentages3]

test_stat3, p_value3 = chisquare(observed3, expected3)

test_stat3, p_value3

## For jockey gender: 

#### Overall percentage of men and women in races where both are represented:

In [ ]:
expected3#Drops races where there is only one jockey gender, meaning that the other gender can't win
jockey_sex_population = df_grouped.jockey_sex.value_counts(normalize=True).where(lambda x: x != 1).dropna()

#Finds mean percent of jockey genders in races
#However, this method drops any race where there is a NaN value 
jockey_sex_population.mean(level=3)

In [ ]:
jockey_sex_population_indices = jockey_sex_population.index

jockey_sex_population_cleaned = df_cleaned.loc[jockey_sex_population_indices].drop_duplicates()

#### Finding the total a different way:

In [ ]:
jockey_sex_population_cleaned.jockey_sex.value_counts(normalize=True)
#This amount is the sum of all 'male' and 'female' jockeys added together and THEN 'normalized'

#### Win percentage of those races:

In [ ]:
#Isolates wins in races with both jockey genders represented
jockey_sex_win_indices = jockey_sex_population_cleaned.position_again.apply(lambda x: x == 1)

jockey_sex_population_cleaned[jockey_sex_win_indices].jockey_sex.value_counts(normalize=True, dropna=False)

In [ ]:
#Finding the percent difference between win and total
jockey_sex_pop = jockey_sex_population_cleaned.jockey_sex.value_counts(normalize=True, dropna=False).values

jockey_sex_win = jockey_sex_population_cleaned[jockey_sex_win_indices].jockey_sex.value_counts(normalize=True, dropna=False).values

jockey_sex_percent_difference = (jockey_sex_win - jockey_sex_pop)/jockey_sex_pop

In [ ]:
index = ['Men','Women']

df4 = pd.DataFrame({'Total Proportion': jockey_sex_pop,'Win Proportion': jockey_sex_win , 'Percent Difference': jockey_sex_percent_difference}, index=index)

ax = df4.plot.bar(rot=0, title='The Significance of Jockey Gender')

### Using a 2-proportion z-test, I find that jockey gender is significant with a p-value of 2.3E-30
#### (There is currently a bug with the statsmodels library concering compatibility with scipy, so I used a scientific calculator)